<a href="https://colab.research.google.com/github/VIPERtips/BayMax-MD/blob/main/DiseasePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import numpy as np
from PIL import Image, ImageDraw
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18
from sklearn.model_selection import train_test_split

# 1. Create dataset folders
os.makedirs("dataset/healthy", exist_ok=True)
for disease in ["rust", "blight", "powdery_mildew", "leaf_spot"]:
    os.makedirs(f"dataset/{disease}", exist_ok=True)

# 2. Generate synthetic images (64x64)
def generate_healthy_leaf_image(size=(64,64)):
    base = np.random.normal(loc=120, scale=10, size=(size[0], size[1], 3))
    base[:, :, 1] += 50  # greener tint
    base = np.clip(base, 0, 255).astype(np.uint8)
    return Image.fromarray(base)

def generate_diseased_leaf_image(size=(64,64), disease="rust"):
    img = generate_healthy_leaf_image(size)
    draw = ImageDraw.Draw(img)
    num_spots = random.randint(5, 15)
    for _ in range(num_spots):
        x = random.randint(5, size[0]-10)
        y = random.randint(5, size[1]-10)
        r = random.randint(3, 7)
        color_map = {
            "rust": (180, 90, 0),
            "blight": (50, 30, 30),
            "powdery_mildew": (230, 230, 230),
            "leaf_spot": (0, 0, 0),
        }
        color = color_map.get(disease, (0,0,0))
        draw.ellipse((x-r, y-r, x+r, y+r), fill=color)
    return img

# 3. Generate and save dataset
num_samples_per_class = 250
classes = ["healthy", "rust", "blight", "powdery_mildew", "leaf_spot"]

print("Generating dataset...")

for cls in classes:
    for i in range(num_samples_per_class):
        if cls == "healthy":
            img = generate_healthy_leaf_image()
        else:
            img = generate_diseased_leaf_image(disease=cls)
        img.save(f"dataset/{cls}/{cls}_{i}.png")

print("Dataset generated!")

# 4. Dataset Loader class
class LeafDiseaseDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# 5. Prepare file paths and labels
all_image_paths = []
all_labels = []

label_map = {cls:i for i, cls in enumerate(classes)}

for cls in classes:
    files = os.listdir(f"dataset/{cls}")
    for f in files:
        all_image_paths.append(f"dataset/{cls}/{f}")
        all_labels.append(label_map[cls])

# 6. Train-test split
train_paths, val_paths, train_labels, val_labels = train_test_split(
    all_image_paths, all_labels, test_size=0.2, stratify=all_labels, random_state=42
)

# 7. Transforms
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

# 8. Datasets and loaders
train_dataset = LeafDiseaseDataset(train_paths, train_labels, transform=transform)
val_dataset = LeafDiseaseDataset(val_paths, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# 9. Model - Use pretrained ResNet18 (small, good baseline)
model = resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, len(classes))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 10. Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 11. Training loop
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f}")

# 12. Validation & test inference function
def predict_image(img_path):
    model.eval()
    img = Image.open(img_path).convert("RGB")
    img_t = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img_t)
        probs = torch.softmax(output, dim=1)
        conf, pred = torch.max(probs, 1)
        return classes[pred.item()], conf.item()

# 13. Test a random image
test_img = val_paths[0]
pred_class, confidence = predict_image(test_img)
print(f"Test Image: {test_img}\nPredicted: {pred_class} with confidence {confidence:.2f}")


Generating dataset...
Dataset generated!


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/5 - Loss: 0.1783
Epoch 2/5 - Loss: 0.1153
Epoch 3/5 - Loss: 0.1901
Epoch 4/5 - Loss: 0.1065
Epoch 5/5 - Loss: 0.1601
Test Image: dataset/rust/rust_189.png
Predicted: rust with confidence 0.99


In [2]:
import torch

torch.save(model.state_dict(), "leaf_disease_model.pkl")
print("Model saved as leaf_disease_model.pkl")


Model saved as leaf_disease_model.pkl


In [4]:
from google.colab import files
files.download("leaf_disease_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>